In [1]:
import spacy
from spacy import displacy
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import warnings
warnings.filterwarnings("ignore")

/home/sever_162/anaconda3/envs/pytorch_bert_T_allemNLP/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SRL_model_path = "https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz"

Predictor =Predictor.from_path(SRL_model_path)

#predictions = Predictor.predict(sentence = "I can have a apple." )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
predictions

{'verbs': [{'verb': 'can',
   'description': 'I [V: can] have a apple .',
   'tags': ['O', 'B-V', 'O', 'O', 'O', 'O']},
  {'verb': 'have',
   'description': '[ARG0: I] [ARGM-MOD: can] [V: have] [ARG1: a apple] .',
   'tags': ['B-ARG0', 'B-ARGM-MOD', 'B-V', 'B-ARG1', 'I-ARG1', 'O']}],
 'words': ['I', 'can', 'have', 'a', 'apple', '.']}

In [5]:
for i in predictions["verbs"]:
    print(i)
    print(":" +i["description"])

{'verb': 'can', 'description': 'I [V: can] have a apple .', 'tags': ['O', 'B-V', 'O', 'O', 'O', 'O']}
:I [V: can] have a apple .
{'verb': 'have', 'description': '[ARG0: I] [ARGM-MOD: can] [V: have] [ARG1: a apple] .', 'tags': ['B-ARG0', 'B-ARGM-MOD', 'B-V', 'B-ARG1', 'I-ARG1', 'O']}
:[ARG0: I] [ARGM-MOD: can] [V: have] [ARG1: a apple] .


In [3]:
import pandas as pd

itrust = pd.read_csv("./study_data/iTrust_Text2Policy.csv")

itrust.info()
Sen_type = itrust.type
ID = itrust.ID
AC_label = itrust.AccessControl
AC_X_label = itrust.XiaoACP
AC_X_label =AC_X_label.fillna(value='FALSE')
Sentence = itrust.Sentence

Sentence =Sen_type + "," + Sentence
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   type               471 non-null    object
 1   numNodessize       471 non-null    int64 
 2   NumberOfWords      471 non-null    int64 
 3   ID                 471 non-null    int64 
 4   Sentence           471 non-null    object
 5   AccessControl      471 non-null    bool  
 6   XiaoACP            418 non-null    object
 7   ModelVerb          418 non-null    object
 8   PassiveTo          418 non-null    object
 9   AccessExpression   418 non-null    object
 10  AbilityExpression  418 non-null    object
 11  AccessControl.1    418 non-null    object
 12  Unnamed: 12        264 non-null    object
 13  Unnamed: 13        157 non-null    object
 14  Unnamed: 14        76 non-null     object
 15  Unnamed: 15        50 non-null     object
 16  Unnamed: 16        36 non-null     object
 1

In [4]:

def append_data(id,verb,descrip):
        current_id.append(id)
        if verb == '*':
                current_verb.append("NULL")
                current_description.append("NULL")
                return 0
        else:
                current_verb.append(verb)
                current_description.append(descrip)
                return 0

current_id = []
current_verb = []
current_description = []
current_sen = []

#Do prediction 
for i in ID:
        predictions = Predictor.predict(sentence = Sentence[i-1] )
        
        if predictions["verbs"] == []:
                append_data(i,'*','*')
                current_sen.append(Sentence[i-1])
                #print(i)
                #print("NO verbs")
        else:
                for verb in predictions["verbs"]:
                        append_data(i,verb["verb"],verb["description"])
                        current_sen.append(Sentence[i-1])
                        #print(i)
                        #print(verb["verb"])
                        #print(":" +verb["description"])
  
print((len(current_id)+len(current_description)+len(current_sen)+len(current_verb))/4)  

KeyboardInterrupt: 

In [31]:
current_SRL_data = pd.DataFrame({ "id":current_id,"Sentence":current_sen,"verb":current_verb,"description":current_description})
current_SRL_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1151 entries, 0 to 1150
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1151 non-null   int64 
 1   Sentence     1151 non-null   object
 2   verb         1151 non-null   object
 3   description  1151 non-null   object
dtypes: int64(1), object(3)
memory usage: 36.1+ KB


In [32]:
#trust = pd.read_csv('iTrust_Text2Policy.csv')

type= []
AC = []
AC_x =[]

Sen_type = itrust.type
ID = itrust.ID
AC_label = itrust.AccessControl
AC_X_label = itrust.XiaoACP
AC_X_label =AC_X_label.fillna(value='False')
Sen = itrust.Sentence

for id in  current_SRL_data.id:
    type.append(Sen_type[id-1])
    AC.append( str(AC_label[id-1]))
    AC_x.append(str(AC_X_label[id-1]))
    #print(id)
    #print(AC_label[id-1])
    #print(Sen_type[id-1])
    #print(Sen_type[id-1])
    #print(Sen[id-1])


In [33]:
combime= {"type":type,"AccessControl":AC,"Xiao_AC":AC_x}
data =pd.DataFrame(combime)
new = pd.concat([ current_SRL_data,data],axis=1)
new.head(10)

,id,Sentence,verb,description,type,AccessControl,Xiao_AC
0,1,UC1 Create and Disable Patients Use Case,NULL,NULL,title,False,False
1,2,An HCP creates patients.,creates,[ARG0: An HCP] [V: creates] [ARG1: patients] .,normal,True,False
2,3,The HCP enters a patient as a new user of system.,enters,[ARG0: The HCP] [V: enters] [ARG1: a patient] ...,normal,True,False
3,4,Only the name and email are provided by the HCP.,are,Only the name and email [V: are] provided by t...,normal,True,False
4,4,Only the name and email are provided by the HCP.,provided,[ARG1: Only the name and email] are [V: provid...,normal,True,False
5,5,The HCP can edit the patient according to data...,can,The HCP [V: can] edit the patient according to...,normal,True,True
6,5,The HCP can edit the patient according to data...,edit,[ARG0: The HCP] [ARGM-MOD: can] [V: edit] [ARG...,normal,True,True
7,5,The HCP can edit the patient according to data...,according,The HCP can edit the patient [V: according] to...,normal,True,True
8,6,Patient MID should be the number.,should,Patient MID [V: should] be the number .,normal,False,False
9,6,Patient MID should be the number.,be,[ARG1: Patient MID] [ARGM-MOD: should] [V: be]...,normal,False,False


In [34]:
order = ["id","type","AccessControl","Xiao_AC","Sentence","verb","description"]
new = new.reindex(columns=order)
new.info()

data_name = ["CyberChair","Collected_ACP","IBM_Course_Management","Itrust_ACRE","iTrust_Text2Policy"]
new.to_csv("iTrust_Test2Policy_convert.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1151 entries, 0 to 1150
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1151 non-null   int64 
 1   type           1151 non-null   object
 2   AccessControl  1151 non-null   object
 3   Xiao_AC        1151 non-null   object
 4   Sentence       1151 non-null   object
 5   verb           1151 non-null   object
 6   description    1151 non-null   object
dtypes: int64(1), object(6)
memory usage: 63.1+ KB
